# MAST30034 Project 2 Group 1 Summary Notebook

## 1. Data collection

### Data sources

#### External datasets
- **ABS Census data:**
- **Crime data:**
- **Population data:**
- **Public transport data:** Downloaded from PTV General Transit Feed Specification (GTFS) Data. As of September 2025.
- **School data:**

#### Given datasets
- **Rental property listings:** Scraped from domain.com.au. As of September 2025.
- **Moving annual median rent by suburb and town:** Victoria Government Families, Fairness and Housings Rental report. As of March quarter 2025.

**Assumptions and limitations:**

The rental dataset reflects only observed/scraped listings and may not necessarily be representative of all rental properties available within Victoria.
Additionally, using several external datasets (resulting in 57 total features) may introduce potential multicollinearity between features (e.g. number of schools vs suburb population). Thus, the model may capture correlations rather than causal effects. However, we aim to address this through feature selection and feature importance.

## 2. Dataset building

The 5 external datasets were processed and cleaned to be merged with the rental property listings dataset.

This means that for each rental listing, there will be rental specific attributes (e.g. number of bedrooms, number of schools within 2km), as well as suburb level attributes (e.g. suburb population, suburb crime index).

This gives us 57 initial attributes/features, which need to be further cleaned, processed, filtered and selected to analyse rental prices.

In [1]:
import pandas as pd
merged_dataset = pd.read_csv("../data/processed/real_estate/vic_rentals_all_enriched.csv")
merged_dataset.columns

Index(['listing_id', 'suburb', 'postcode', 'weekly_rent', 'bond',
       'available_date', 'date_listed', 'days_listed', 'bedrooms', 'bathrooms',
       'carspaces', 'property_type', 'address', 'lat', 'lon', 'photo_count',
       'video_count', 'floorplans_count', 'virtual_tour', 'primary_type',
       'secondary_type', 'agency', 'agent_names', 'land_area',
       'num_metro_bus_stops', 'num_metro_tram_stops', 'num_metro_train_stops',
       'num_regional_bus_stops', 'num_regional_train_stops', 'num_schools_2km',
       'Median_age_persons', 'Median_mortgage_repay_monthly',
       'Median_tot_prsnl_inc_weekly', 'Median_rent_weekly',
       'Median_tot_fam_inc_weekly', 'Average_num_psns_per_bedroom',
       'Median_tot_hhd_inc_weekly', 'Average_household_size',
       'Owner occupied (%)', 'Mortgage (%)', 'Total rented (%)',
       'Other tenure (%)', 'Unemployment', 'post_gradutae (%)',
       'Graduate_diploma_certificate(%)', 'Bachelor (%)',
       'Advanced_&_Diploma (%)', 'Certific

## 3. Feature Engineering

### Handling missing values

With the initial merged dataset, need to deal with any possible nan values per feature. This follows a 2 step process:

**1. Mean imputation:** We create a lookup dictionary that is grouped by the property's suburb, property_type, bedrooms, and bathrooms and impute the feature's missing value according to the mean-aggregated dictionary value. Next, we create a relaxed version of this lookup dictionary on 'property_type', 'bedrooms' and use a similar pattern to impute more nans. This is done for the "weekly_rent" and "carspaces" features, which have the among the highest missing values. 

**2. Listwise deletion:** After imputing nans, there is significantly less remaining missing values so just drop them.

Note: land_area was simply dropped and remove despite being a potential useful feature as it had too many missing values i.e 12329/12331

**Assumptions and limitations**
- Assumes features are missing at random and not systematically biased
- Mean imputation may be too simple a method for imputation so may ignore natural variance in features. Though this is aimed to be addressed through the use of lookup dictionaries.

In [2]:
def find_nans(data):
    missing_list = [(col, data[col].isnull().sum()) for col in data.columns]
    non_nans = [(col, cnt) for col, cnt in missing_list if cnt != 0]
    return sorted(non_nans, key=lambda x: x[1], reverse=True)  # sort by column name

In [3]:
data = data.drop(columns=["land_area", "SAL_NAME21", 'suburb', 'bond'])
print(find_nans(data))

NameError: name 'data' is not defined

### Outlier detection

These numerical features were considered to determine outliers: ['weekly_rent', 'bedrooms', 'bathrooms', 'carspaces', 'num_metro_bus_stops', 'num_metro_tram_stops', 'num_schools_2km', 'incidents_recorded']

We assume that extreme values are unrepresentative, which should be a valid assumption as there are only 27 rental properties above $3000, which should have little impact on model performance.

We used $3000 as the as upper limit for the rental prices of houses in the Vic-Gov website is $2885. (https://www.housing.vic.gov.au/what-does-rent-cost-victoria)

In [ ]:
#Find how many 0 weekly_rent values there are
zero_rent_count = (data["weekly_rent"] == 0).sum()
print("Zero rent count:", zero_rent_count)

#Find how many high outlier weekly_rent values there are i.e above 3000
highoutlier_rent_count = (data["weekly_rent"] >= 3000).sum()
print("High outlier rent count:", highoutlier_rent_count)

#Find how many data points with 50 or more bedrooms
high_bedroom_count = (data["bedrooms"] >= 50).sum()
print("High bedroom count:", high_bedroom_count)

Zero rent count: 16
High outlier rent count: 27
High bedroom count: 1


Surprisingly, not many outliers were detected. Removing these outliers gives the following distribution for the numerical values:

In [ ]:
#Remove outliers rows
data = data[(data["weekly_rent"] > 0) & (data["weekly_rent"] <= 3000) & (data["bedrooms"] < 50)]
#Looking at numerical variables
data[['weekly_rent', 'bedrooms', 'bathrooms', 'carspaces', 'num_metro_bus_stops', 'num_metro_tram_stops', 'num_schools_2km', 'incidents_recorded']].describe()

,weekly_rent,bedrooms,bathrooms,carspaces,num_metro_bus_stops,num_metro_tram_stops,num_schools_2km,incidents_recorded
count,12018.000000,12018.000000,12018.000000,12018.000000,12018.000000,12018.000000,12018.000000,12018.000000
mean,621.903977,2.720835,1.587203,1.626061,61.837910,20.948910,8.061075,13259.928384
std,249.176926,1.081993,0.629443,0.946937,43.221829,35.017988,4.798088,5828.544925
min,33.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,77.000000
25%,490.000000,2.000000,1.000000,1.000000,22.000000,0.000000,4.000000,9525.000000
50%,560.000000,3.000000,2.000000,1.000000,66.000000,0.000000,8.000000,13140.500000
75%,683.000000,4.000000,2.000000,2.000000,96.000000,35.000000,12.000000,17495.333333
max,3000.000000,11.000000,12.000000,22.000000,183.000000,127.000000,23.000000,34620.000000


Finally, after encoding categorical variables (extracting time features from 'available_date'), our dataset is finally cleaned and ready to be analysed to determine important features for predicting rental prices.

## 4. Modelling Feature Importance

Models: Random Forest Regressor and GX boost were selected with their ability to capture complex relationships in data and a useful feature importance function to help understand which features are were most important in model predictions. 

### Feature Engeering And Encoding

Time data was feature engineered to hour, day and month; and then encoded using cyclic encoding to help model capture potential seasonal change in rent prices.

Frequency encoding was used for non-numerical features i.e postcode, property_type and agency. 

In [ ]:
#Feature Engineering time data
data['available_date'] = pd.to_datetime(data['available_date'], errors='coerce')
data['available_day'] = data['available_date'].dt.day
data['available_month'] = data['available_date'].dt.month   
data['available_year'] = data['available_date'].dt.year
data = data.drop(columns=['available_date'])

In [ ]:
# Encode month cyclically 
data['month_sin'] = np.sin(data['available_month'] / 12 * 2 * np.pi)
data['month_cos'] = np.cos(data['available_month'] / 12 * 2 * np.pi)
data = data.drop(columns=['available_month'])

#Encode day cyclically
data['day_sin'] = np.sin(data['available_day'] / 31 * 2 * np.pi)
data['day_cos'] = np.cos(data['available_day'] / 31 * 2 * np.pi)
data = data.drop(columns=['available_day'])

#Frequency encoding for Non-numericeal columns
post_freq = data['postcode'].value_counts(normalize=True)
data['postcode'] = data['postcode'].map(post_freq)
property_freq = data['property_type'].value_counts(normalize=True)
data['property_type'] = data['property_type'].map(property_freq)
agency_freq = data['agency'].value_counts(normalize=True)
data['agency'] = data['agency'].map(agency_freq)

#### Limitations and Assumptions

We assume that postcode, propety_type and agency have certain catergories(e.g a common postcode, popular property type,etc.) that can influence weekly rent prices. 

Cycle encodinng treats and months and day pattern as if patterns always repeat identically, training only on 2025 might cause bias in the predictions of data for a different year. This shouldn't be an issue as this data set will not be used in forcasting.

### Data leakage

Certain features in the data such as 'median_rent_weekly', 'median_morgage_repay_monthly' and 'bond' can cause data leakage and hence removed. (bond was remove earlier in preprocessing)

In [ ]:
#Drop lat and long for modeling
data = data.drop(columns=['lat', 'lon', 'Median_rent_weekly', 'Median_mortgage_repay_monthly'])

All data was rescaled using the Standardize Scalar method

In [ ]:
#Standardize scalar, resacling all data. (can target specific columns if needed)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Findings

Feature Selection was conducted using Mutual Information(MI) as we hav 42 features. 20 Features were selected from the 42.

Results show that models without feature selection performes significantly better with average r^2 of 0.7 and average MAE of 70 compared to mdoels with feature selection with average r^2  0.25 and average MAE of 126.5.

Cause: likely due to MI only capturing univeriate relationships between one variable vs target variable while our data contains strong multivariable interactions; hence MI fails to compare them and discards useful features.

In [ ]:
# Compute MI
mi = mutual_info_regression(X, y, discrete_features="auto", random_state=0)
mi_scores = pd.Series(mi, index=X.columns).sort_values(ascending=False)

k = 20
selected_features = mi_scores.head(k).index.tolist()

X_selection = X[selected_features]
X_selection

## 5. Liveability & affordability



### Components that go into livability score of a suburb are: 
- number of schools  
- number of train, bus and tram stops 
- crime index as normalised (0-100) score of total offenses recorded within the suburb, where below 20 is considered low crime

To quanitfy the livability we assigned weights to the components above, the assumption was that crime is the most important indicator of suburb's quality hence its weight of 0.4. Number of schools is the second most important factor with weight 0.3, then number of public transport stops with overall weight of 0.3. 

### Afordability calculation 

Affordability was quantified as ratio of weekly rent to median weekly household income in a suburb


In [1]:
# Import libraries
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# Load the enriched suburbs dataset
df = pd.read_csv('../data/curated/cleaned_real_estate_data.csv')
df.head()

# Aggregate to postcode level (use median for rents/coords, median/mean for counts)
agg = df.groupby('postcode').agg(
    median_tot_hhd_inc_weekly = ('Median_tot_hhd_inc_weekly','median'),
    num_schools_2km = ('num_schools_2km', 'median'),
    num_metro_train_stops = ('num_metro_train_stops', 'median'),
    num_metro_tram_stops = ('num_metro_tram_stops', 'median'),
    num_metro_bus_stops = ('num_metro_bus_stops', 'median'),
    crime_index = ('crime_index', 'median'),
    weekly_rent = ('weekly_rent', 'median'),
    lat = ('lat', 'median'),
    lon = ('lon', 'median'),
    count_rows = ('postcode', 'count')
).reset_index()

# Scale features (higher = better). Invert crime & rent.
scaler = MinMaxScaler()

def norm(series):
    return scaler.fit_transform(series.values.reshape(-1,1)).ravel()

# Livability components (higher = better). Invert crime.
agg['schools_norm'] = norm(agg['num_schools_2km'])
agg['train_norm']   = norm(agg['num_metro_train_stops'])
agg['tram_norm']    = norm(agg['num_metro_tram_stops'])
agg['bus_norm']     = norm(agg['num_metro_bus_stops'])
agg['crime_norm']   = 1 - norm(agg['crime_index'])


# Composite scores (chosen metrics / weights)
# Livability weights: schools 0.30, train 0.10, tram 0.10, bus 0.10, crime 0.40
agg['livability_score'] = (
    0.30 * agg['schools_norm'] +
    0.10 * agg['train_norm'] +
    0.10 * agg['tram_norm'] +
    0.10 * agg['bus_norm'] +
    0.40 * agg['crime_norm']
)

# Affordability using median_tot_hhd_inc_weekly:
# rent_to_income = weekly_rent / median_tot_hhd_inc_weekly (lower = more affordable)
agg['rent_to_income'] = agg['weekly_rent'] / agg['median_tot_hhd_inc_weekly']
agg['affordability_score'] = 1 - norm(agg['rent_to_income'])  # higher = more affordable

# Ranks
agg['livability_rank'] = agg['livability_score'].rank(ascending=False, method='min')
agg['affordability_rank'] = agg['affordability_score'].rank(ascending=False, method='min')

# Top lists
top_livable = agg.sort_values('livability_score', ascending=False).head(20)[
    ['postcode','livability_score','livability_rank','affordability_score']
]
top_affordable = agg.sort_values('affordability_score', ascending=False).head(20)[
    ['postcode','affordability_score','affordability_rank','livability_score']
]

print("Top 20 most livable (postcode proxy):")
print(top_livable.to_string(index=False))
print("\nTop 20 most affordable (postcode proxy):")
print(top_affordable.to_string(index=False))

import geopandas as gpd
import folium
from matplotlib import cm, colors

# Load the shapefile for postcode boundaries
postcode_gdf = gpd.read_file('../../data/landing/boundaries/SA2_2021_AUST_GDA2020.shp')

# Center on Victoria (approximate center)
vic_center = [-37.0, 144.5]
m_livability = folium.Map(location=vic_center, zoom_start=7, tiles='cartodbpositron')
m_affordability = folium.Map(location=vic_center, zoom_start=7, tiles='cartodbpositron')

# Select top 20 only
top_n = 20
top_livable = agg.nlargest(top_n, 'livability_score').reset_index(drop=True)
top_affordable = agg.nlargest(top_n, 'affordability_score').reset_index(drop=True)

# Colormaps and normalization (use top-20 range so colours are meaningful)
cmap_liv = cm.get_cmap('YlGn')
cmap_aff = cm.get_cmap('YlOrRd')

liv_min, liv_max = top_livable['livability_score'].min(), top_livable['livability_score'].max()
aff_min, aff_max = top_affordable['affordability_score'].min(), top_affordable['affordability_score'].max()

# Plot top 20 livable
for i, row in top_livable.iterrows():
    if np.isnan(row['lat']) or np.isnan(row['lon']):
        continue
    val = 0.5
    if liv_max > liv_min:
        val = (row['livability_score'] - liv_min) / (liv_max - liv_min)
    hexcol = colors.to_hex(cmap_liv(val))
    radius = 7
    # highlight the best (first row from nlargest)
    if i == 0:
        radius = 12
        hexcol = '#0000FF'  # blue for best
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,
        fill=True,
        fill_opacity=0.9,
        color=hexcol,
        fill_color=hexcol,
        tooltip=f"Postcode: {row['postcode']}<br>Livability: {row['livability_score']:.3f}<br>Rank: {int(row['livability_rank']) if 'livability_rank' in row else 'N/A'}"
    ).add_to(m_livability)
    
display(m_livability)


Top 20 most livable (postcode proxy):
 postcode  livability_score  livability_rank  affordability_score
     3126          0.733811              1.0             0.745382
     3123          0.731158              2.0             0.792613
     3183          0.717356              3.0             0.765768
     3162          0.712354              4.0             0.759937
     3068          0.688550              5.0             0.732063
     3185          0.673369              6.0             0.777010
     3122          0.666856              7.0             0.767968
     3144          0.656467              8.0             0.732655
     3204          0.655334              9.0             0.684032
     3103          0.654143             10.0             0.680877
     3189          0.654015             11.0             0.724151
     3142          0.653293             12.0             0.748000
     3124          0.652378             13.0             0.771147
     3146          0.651324           

ModuleNotFoundError: No module named 'folium'

Models 
Key features: bathrooms, bedrooms and Bachelor (%) were signficantly more important than every other variable.

## 5. Rent Forecasting

### Findings 



In [3]:
import pandas as pd 

houses = pd.read_csv("../data/outputs/houses_growth.csv")

units = pd.read_csv("../data/outputs/apartments_growth.csv")

overall = pd.read_csv("../data/outputs/overall_growth.csv")

## 6. Limitations